# User Groups

In [1]:
# import required packages
import pandas as pd
from collections import defaultdict
from tqdm import tqdm

In [2]:
# config
ds_name = 'MovieLens'
# ds_acronym = 'BX'
# ds_core = '5Rate'
# ds_feedback = 'Explicit'

In [3]:
# loading the Book-Crossing dataset
dataset = pd.read_csv(f"datasets/{ds_name}/{ds_name}_train.txt", sep="\t", names=['uid', 'iid', 'preference'])

In [4]:
dataset.head(5)

,uid,iid,preference
0,0,0,4
1,0,4,5
2,0,18,4
3,0,22,3
4,0,24,5


In [5]:
# get user distribution
user_dist = dataset['uid'].value_counts()
num_users = len(user_dist)
print('No. users: ' + str(num_users))
print('Mean books per user: ' + str(user_dist.mean()))
print('Min books per user: ' + str(user_dist.min()))
print('Max books per user: ' + str(user_dist.max()))

No. users: 943
Mean books per user: 73.70095440084836
Min books per user: 10
Max books per user: 461


In [6]:
# get item distribution
item_dist = dataset['iid'].value_counts()
num_items = len(item_dist)
print('No. items: ' + str(num_items))

No. items: 1349


In [7]:
# get top items
top_fraction = 0.2
num_top = int(top_fraction * num_items)
top_item_dist = item_dist[:num_top]
print('No. top items: ' + str(num_top))

No. top items: 269


In [8]:
top_item_set = set(top_item_dist.keys())

In [9]:
# explicit
user_items = defaultdict(set)
for eachline in tqdm(dataset.iterrows()):
    uid, iid, pref = eachline[1][0], eachline[1][1], eachline[1][2] 
    uid, iid = int(uid), int(iid)
    if uid in user_items.keys():
        user_items[uid].add(iid)
    else:
        user_items[uid] = {iid}

69500it [00:09, 7640.73it/s]


In [10]:
user_pop_item_ratio = {}

for user, items in tqdm(user_items.items()):
    if user not in user_pop_item_ratio.keys():
        user_pop_item_ratio[user] = (len(set(items) & set(top_item_set))) / len(set(items))

100%|██████████| 943/943 [00:00<00:00, 68038.75it/s]


In [11]:
sorted_user_pop_item_ratio = {k: v for k, v in sorted(user_pop_item_ratio.items(), key=lambda item: item[1], reverse=True)}

In [12]:
num_user = len(sorted_user_pop_item_ratio)
num_top_users = int(top_fraction * num_users)
num_top_users

188

In [13]:
# Files to save users ids based on the number of checkins
unpopular_users = open(f'datasets/{ds_name}/user-groups/unpopular_users.txt', 'w')
# inactive_users.write('user_id' + '\n')

diverse_users = open(f'datasets/{ds_name}/user-groups/diverse_users.txt', 'w')
# medium_users.write('user_id' + '\n')

popular_users = open(f'datasets/{ds_name}/user-groups/popular_users.txt', 'w')
# active_users.write('user_id' + '\n')

In [14]:
for uid in list(sorted_user_pop_item_ratio.keys())[:num_top_users]:
    popular_users.write(str(uid) + '\n')
popular_users.close()

In [15]:
for uid in list(sorted_user_pop_item_ratio.keys())[num_top_users:len(sorted_user_pop_item_ratio) - num_top_users]:
    diverse_users.write(str(uid) + '\n')
diverse_users.close()

In [16]:
for uid in list(sorted_user_pop_item_ratio.keys())[len(sorted_user_pop_item_ratio) - num_top_users:len(sorted_user_pop_item_ratio)]:
    unpopular_users.write(str(uid) + '\n')
unpopular_users.close()

In [17]:
# file closing
unpopular_users.close()
diverse_users.close()
popular_users.close()